### Corrected and Improved Code (Cell 1: Setup)

This first cell now not only sets up the path for your `src` modules but also defines key directory variables (`DATA_DIR`, `ROOT_DIR`) that you can use throughout your notebook.


In [1]:
import sys
from pathlib import Path
import pandas as pd
import os
from IPython.display import display, Markdown  # Assuming you use these for display

# --- 1. PANDAS OPTIONS (No change) ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)

# --- 2. IPYTHON AUTORELOAD (No change) ---
%load_ext autoreload
%autoreload 2
%matplotlib widget

# --- 3. ROBUST PATH CONFIGURATION (MODIFIED) ---

# Get the current working directory of the notebook
NOTEBOOK_DIR = Path.cwd()

# Find the project ROOT directory by going up from the notebook's location
# This is robust and works even if you move the notebook deeper.
ROOT_DIR = NOTEBOOK_DIR.parent.parent

# Define key project directories relative to the ROOT
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
# You could also define an output directory here if needed
OUTPUT_DIR = ROOT_DIR / 'output'

# Add the 'src' directory to the Python path so you can import 'utils'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- 4. VERIFICATION (IMPROVED) ---
print(f"✅ Project Root Directory: {ROOT_DIR}")
print(f"✅ Notebook Directory: {NOTEBOOK_DIR}")
print(f"✅ Source Directory (for utils): {SRC_DIR}")
print(f"✅ Data Directory (for input): {DATA_DIR}")

# Verify that the key directories exist. This helps catch path errors early.
assert ROOT_DIR.exists(), f"ROOT directory not found at: {ROOT_DIR}"
assert SRC_DIR.exists(), f"Source directory not found at: {SRC_DIR}"
assert DATA_DIR.exists(), f"Data directory not found at: {DATA_DIR}"

# --- 5. IMPORT YOUR CUSTOM MODULE ---
import utils
print("\n✅ Successfully imported 'utils' module.")

✅ Project Root Directory: c:\Users\ping\Files_win10\python\py311\stocks
✅ Notebook Directory: c:\Users\ping\Files_win10\python\py311\stocks\notebooks_rank\_working
✅ Source Directory (for utils): c:\Users\ping\Files_win10\python\py311\stocks\src
✅ Data Directory (for input): c:\Users\ping\Files_win10\python\py311\stocks\data

✅ Successfully imported 'utils' module.


### Corrected Code (Cell 2: Execution)

Now, in your second cell, you simply use the `DATA_DIR` variable we defined in the setup cell. This removes the fragile relative path `..\data`.

In [3]:
# To get ALL matching files, sorted by recency
file_list = utils.get_recent_files(
    directory_path=DATA_DIR,
    extension='parquet',
    prefix=None,
    contains_pattern='df_finviz_merged_stocks_etfs',
    count=None
)

# Print the file_list
print(f'\nfile_list (len(file_list): {len(file_list)}:')
for i, f, in enumerate(file_list):
    print(f'{i}: {f}')



file_list (len(file_list): 54:
0: 2025-07-11_df_finviz_merged_stocks_etfs.parquet
1: 2025-07-10_df_finviz_merged_stocks_etfs.parquet
2: 2025-07-09_df_finviz_merged_stocks_etfs.parquet
3: 2025-07-08_df_finviz_merged_stocks_etfs.parquet
4: 2025-07-07_df_finviz_merged_stocks_etfs.parquet
5: 2025-07-03_df_finviz_merged_stocks_etfs.parquet
6: 2025-06-06_df_finviz_merged_stocks_etfs.parquet
7: 2025-06-05_df_finviz_merged_stocks_etfs.parquet
8: 2025-06-04_df_finviz_merged_stocks_etfs.parquet
9: 2025-06-03_df_finviz_merged_stocks_etfs.parquet
10: 2025-06-02_df_finviz_merged_stocks_etfs.parquet
11: 2025-05-30_df_finviz_merged_stocks_etfs.parquet
12: 2025-05-29_df_finviz_merged_stocks_etfs.parquet
13: 2025-05-28_df_finviz_merged_stocks_etfs.parquet
14: 2025-05-27_df_finviz_merged_stocks_etfs.parquet
15: 2025-05-23_df_finviz_merged_stocks_etfs.parquet
16: 2025-05-22_df_finviz_merged_stocks_etfs.parquet
17: 2025-05-21_df_finviz_merged_stocks_etfs.parquet
18: 2025-05-20_df_finviz_merged_stocks_etf

In [4]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

def create_rank_history_df(file_list, data_dir):
    """
    Reads daily files to compile a history of ranks for each ticker.

    Args:
        file_list (list): A list of sorted parquet filenames.
        data_dir (Path): The directory where files are stored.

    Returns:
        pd.DataFrame: A DataFrame with tickers as the index, dates as columns,
                      and ranks as the values. NaN indicates the ticker was not
                      present on that day.
    """
    daily_ranks_list = []
    for filename in file_list:
        # Extract the date string from the start of the filename
        date_str = filename.split('_')[0]
        
        # Read the daily data file
        df_daily = pd.read_parquet(data_dir / filename)
        
        # Extract the 'Rank' column (it's a Series with tickers as its index)
        ranks_series = df_daily['Rank']
        
        # Rename the Series to the date, which will become the column name
        ranks_series.name = pd.to_datetime(date_str)
        
        daily_ranks_list.append(ranks_series)

    # Concatenate all Series into a single DataFrame, aligning on the ticker index
    df_rank_history = pd.concat(daily_ranks_list, axis=1)
    
    # Sort columns by date just in case the file list wasn't perfectly sorted
    df_rank_history = df_rank_history.sort_index(axis=1)
    
    return df_rank_history

# --- Example Usage ---
# Assuming DATA_DIR is a pathlib.Path to your data folder
# and file_list is your list of filenames.
# DATA_DIR = Path('your_data_directory/')
df_rank_history = create_rank_history_df(file_list, DATA_DIR)

print("Shape of the resulting DataFrame:", df_rank_history.shape)
print(f'\ndf_rank_history:\n{df_rank_history}')

Shape of the resulting DataFrame: (1606, 54)

df_rank_history:
      2025-04-25  2025-04-28  2025-04-29  2025-04-30  2025-05-01  2025-05-02  2025-05-05  2025-05-06  2025-05-07  2025-05-08  2025-05-09  2025-05-12  2025-05-13  2025-05-14  2025-05-15  2025-05-16  2025-05-19  2025-05-20  2025-05-21  2025-05-22  2025-05-23  2025-05-27  2025-05-28  2025-05-29  2025-05-30  2025-06-02  2025-06-03  2025-06-04  2025-06-05  2025-06-06  2025-06-09  2025-06-10  2025-06-11  2025-06-12  2025-06-13  2025-06-16  2025-06-17  2025-06-18  2025-06-19  2025-06-20  2025-06-23  2025-06-24  2025-06-25  2025-06-26  2025-06-27  2025-06-30  2025-07-01  2025-07-02  2025-07-03  2025-07-07  2025-07-08  2025-07-09  2025-07-10  2025-07-11
NVDA         3.0         3.0         3.0         3.0         3.0         3.0         3.0         3.0         3.0         3.0         3.0         3.0         3.0         2.0         2.0         2.0         2.0         2.0         2.0         2.0         2.0         2.0         2.0    

# WOW keep this one
    df_rank_history,
    lookback_days=20,
    recent_days=4,
    min_lookback_rank_improvement=15,
    min_dip_magnitude=0,       # The dip must be at least 15 ranks deep
    min_reversal_improvement=0, # It must have recovered at least 5 ranks from the bottom
    # current_rank_bracket_start=601,
    current_rank_bracket_end=800,

In [9]:
import pandas as pd
import numpy as np

def find_rank_dip_or_reversal_opportunities(
                            df_rank_history, 
                            lookback_days=20, 
                            recent_days=4, 
                            min_lookback_rank_improvement=15,
                            # --- Mode-specific parameters ---
                            min_recent_rank_drop=None,  # Only used in 'dip' mode
                            min_dip_magnitude=None,  # Only used in 'reversal' mode 
                            min_reversal_improvement=None,  # Only used in 'reversal' mode
                            # --- General filters ---
                            current_rank_bracket_start=1, 
                            current_rank_bracket_end=None):
    """
    Scans rank history to find tickers that had a steady uptrend in rank
    followed by a recent significant drop or reversal, within a specified current rank range.
    Adds the ticker's best rank ('best_rank') and its distance from that rank to the output.

    Args:
        df_rank_history (pd.DataFrame): DataFrame with tickers as index, dates as columns, and ranks as values.
        lookback_days (int): The number of days to define the "past performance" period.
        recent_days (int): The number of recent days to check for a "dip".
        min_lookback_rank_improvement (int): The minimum number of ranks a ticker must have improved
                                     during the lookback period to be considered.
        min_recent_rank_drop (int): The minimum number of ranks a ticker must have dropped
                               in the recent period to be considered a "dip".
        current_rank_bracket_start (int): The minimum current rank to be included in the analysis. Defaults to 1.
        current_rank_bracket_end (int, optional): The maximum current rank to be included. If None, no upper limit.

    Returns:
        list: A sorted list of dictionaries, where each dictionary contains information
              about a potential "buy the dip" opportunity.
    """


    # --- Mode Detection and Parameter Validation ---
    # ... (this block is unchanged) ...
    mode = None
    is_dip_param_present = min_recent_rank_drop is not None
    are_reversal_params_present = min_dip_magnitude is not None or min_reversal_improvement is not None
    if is_dip_param_present and are_reversal_params_present:
        dip_args_str = f"min_recent_rank_drop={min_recent_rank_drop}"
        reversal_args = []
        if min_dip_magnitude is not None: reversal_args.append(f"min_dip_magnitude={min_dip_magnitude}")
        if min_reversal_improvement is not None: reversal_args.append(f"min_reversal_improvement={min_reversal_improvement}")
        reversal_args_str = ", ".join(reversal_args)
        error_message = ("Cannot mix modes. You provided parameters for both 'Dip' and 'Reversal' modes.\n"
                         f"  - For 'Dip' mode, you provided: {dip_args_str}\n"
                         f"  - For 'Reversal' mode, you provided: {reversal_args_str}\n"
                         "Please choose one mode and provide only its required arguments.")
        raise ValueError(error_message)
    if is_dip_param_present:
        mode = 'dip'
        print("Running in 'Dip' mode...")
    elif min_dip_magnitude is not None and min_reversal_improvement is not None:
        mode = 'reversal'
        print("Running in 'Reversal' mode...")
    else:
        if are_reversal_params_present:
            provided = []
            if min_dip_magnitude is not None: provided.append(f"min_dip_magnitude={min_dip_magnitude}")
            if min_reversal_improvement is not None: provided.append(f"min_reversal_improvement={min_reversal_improvement}")
            raise ValueError(f"Incomplete 'Reversal' mode parameters. You must provide BOTH 'min_dip_magnitude' and 'min_reversal_improvement'. You only provided: {', '.join(provided)}.")
        else:
            raise ValueError("Must specify parameters for a mode. For 'Dip' mode, provide 'min_recent_rank_drop'. For 'Reversal' mode, provide both 'min_dip_magnitude' and 'min_reversal_improvement'.")

    # --- Guard Clause & Date Setup ---
    # ... (this block is unchanged) ...
    total_days_needed = lookback_days + recent_days
    if len(df_rank_history.columns) < total_days_needed:
        print(f"Error: Not enough data. Need {total_days_needed} days, have {len(df_rank_history.columns)}.")
        return []
    all_dates = df_rank_history.columns
    last_date = all_dates[-1]
    recent_period_start_date = all_dates[-recent_days]
    lookback_period_end_date = all_dates[-(recent_days + 1)]
    lookback_period_start_date = all_dates[-(recent_days + lookback_days)]
    lookback_dates = df_rank_history.loc[:, lookback_period_start_date:lookback_period_end_date].columns
    recent_dates = df_rank_history.loc[:, recent_period_start_date:last_date].columns

    opportunities = []

    # --- Pre-filter tickers ---
    # ... (this block is unchanged) ...
    ranks_on_last_day = df_rank_history[last_date]
    mask_start = (ranks_on_last_day >= current_rank_bracket_start)
    if current_rank_bracket_end is not None:
        mask_end = (ranks_on_last_day <= current_rank_bracket_end)
        tickers_to_scan = df_rank_history[mask_start & mask_end].index
    else:       
        tickers_to_scan = df_rank_history[mask_start].index
    analysis_start_str = pd.to_datetime(lookback_period_start_date).strftime('%Y-%m-%d')
    analysis_end_str = pd.to_datetime(last_date).strftime('%Y-%m-%d')
    print(f"Analyzing {len(tickers_to_scan)} tickers from {analysis_start_str} to {analysis_end_str}, "
          f"currently ranked between {current_rank_bracket_start} and {current_rank_bracket_end or 'max'}...")

    # --- Analysis Loop ---
    for ticker in tickers_to_scan:
        lookback_ranks = df_rank_history.loc[ticker, lookback_dates].dropna()
        recent_ranks = df_rank_history.loc[ticker, recent_dates].dropna()
        
        if len(lookback_ranks) < lookback_days or len(recent_ranks) < recent_days: continue
        if (lookback_ranks.iloc[0] - lookback_ranks.iloc[-1]) < min_lookback_rank_improvement: continue
     
        slope, _ = np.polyfit(np.arange(len(lookback_ranks)), lookback_ranks, 1)
        if slope >= 0: continue

        # --- NEW: Calculate max rank and distance from max ---
        # Combine both periods to find the absolute best rank achieved.
        all_ranks_in_period = pd.concat([lookback_ranks, recent_ranks])
        best_rank = all_ranks_in_period.min()  # Best rank is the smallest number.
        current_rank = recent_ranks.iloc[-1]
        dist_from_best_rank = current_rank - best_rank
        # --- End of new calculation ---

        if mode == 'dip':
            rank_at_start_of_recent = recent_ranks.iloc[0]
            recent_drop = current_rank - rank_at_start_of_recent
            if recent_drop < min_recent_rank_drop: continue
            
            # Add the new fields to the output dictionary
            opportunities.append({
                'ticker': ticker, 
                'current_rank': int(current_rank), 
                'best_rank': int(best_rank),                  #<-- NEW
                'dist_from_best_rank': int(dist_from_best_rank), #<-- NEW
                'previous_rank': int(rank_at_start_of_recent), 
                'recent_drop': int(recent_drop), 
                'lookback_slope': round(slope, 2)
            })

        elif mode == 'reversal':
            rank_at_start_of_recent = recent_ranks.iloc[0]
            dip_rank = recent_ranks.max()
            dip_magnitude = dip_rank - rank_at_start_of_recent
            if dip_magnitude < min_dip_magnitude: continue
            
            reversal_improvement = dip_rank - current_rank
            if reversal_improvement < min_reversal_improvement: continue
            
            # Add the new fields to the output dictionary
            opportunities.append({
                'ticker': ticker, 
                'current_rank': int(current_rank), 
                'best_rank': int(best_rank),                  #<-- NEW
                'dist_from_best_rank': int(dist_from_best_rank), #<-- NEW
                'dip_rank': int(dip_rank), 
                'dip_magnitude': int(dip_magnitude), 
                'reversal_improvement': int(reversal_improvement), 
                'lookback_slope': round(slope, 2)
            })

    # return sorted(opportunities, key=lambda x: x['lookback_slope'])

    # --- MODIFICATION IS HERE ---
    # At the end of the function, before returning, capture the parameters.
    filter_criteria = {
        'lookback_days': lookback_days,
        'recent_days': recent_days,
        'min_lookback_rank_improvement': min_lookback_rank_improvement,
        'min_recent_rank_drop': min_recent_rank_drop,
        'min_dip_magnitude': min_dip_magnitude,
        'min_reversal_improvement': min_reversal_improvement,
        'current_rank_bracket_start': current_rank_bracket_start,
        'current_rank_bracket_end': current_rank_bracket_end,
        'mode': mode # Also capture the detected mode
    }

    sorted_opportunities = sorted(opportunities, key=lambda x: x['lookback_slope'])
    
    # Return both the results and the criteria used to get them.
    return sorted_opportunities, filter_criteria



In [6]:
df_rank_history.iloc[:, :]

,2025-04-25,2025-04-28,2025-04-29,2025-04-30,2025-05-01,2025-05-02,2025-05-05,2025-05-06,2025-05-07,2025-05-08,2025-05-09,2025-05-12,2025-05-13,2025-05-14,2025-05-15,2025-05-16,2025-05-19,2025-05-20,2025-05-21,2025-05-22,2025-05-23,2025-05-27,2025-05-28,2025-05-29,2025-05-30,2025-06-02,2025-06-03,2025-06-04,2025-06-05,2025-06-06,2025-06-09,2025-06-10,2025-06-11,2025-06-12,2025-06-13,2025-06-16,2025-06-17,2025-06-18,2025-06-19,2025-06-20,2025-06-23,2025-06-24,2025-06-25,2025-06-26,2025-06-27,2025-06-30,2025-07-01,2025-07-02,2025-07-03,2025-07-07,2025-07-08,2025-07-09,2025-07-10,2025-07-11
NVDA,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
MSFT,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
AAPL,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
AMZN,4.0,4.0,4.0,4.0,4.0,4.0,6.0,6.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
GOOG,6.0,6.0,5.0,5.0,6.0,5.0,4.0,4.0,6.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,5.0,6.0,5.0,6.0,5.0,6.0,6.0,5.0,5.0,5.0,5.0,6.0,5.0,6.0,5.0,6.0,6.0,6.0,6.0,6.0,5.0,6.0,5.0,5.0,6.0,5.0,6.0,5.0,6.0,5.0,6.0,5.0,6.0,6.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GOLD,433.0,427.0,434.0,434.0,445.0,452.0,447.0,436.0,434.0,448.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ONTO,1336.0,1337.0,NaN,1368.0,1377.0,1347.0,1354.0,1366.0,1358.0,1359.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UCON,1545.0,1545.0,1528.0,1550.0,1551.0,1554.0,1555.0,1557.0,1557.0,1558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RECS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1544.0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
selected_df = df_rank_history.iloc[:, :]       # Get all rows and all columns
# selected_df = df_rank_history.iloc[:, :50]   # Get the first 50 columns 
# selected_df = df_rank_history.iloc[:, -50:]  # Get the last 50 columns 
# selected_df = df_rank_history.iloc[:, :-1]   # Get all columns except the last one

print(f'df_rank_history.shape: {df_rank_history.shape}')
print(f"selected_df.shape:     {selected_df.shape}")

df_rank_history.shape: (1606, 54)
selected_df.shape:     (1606, 54)


In [17]:
lookback_days = 20 
recent_days = 4 
min_lookback_rank_improvement = 15
# --- Mode-specific parameters ---
min_recent_rank_drop = None  # Only used in 'dip' mode
# === Reversal Mode Parameters ===
min_dip_magnitude = 0  # Only used in 'reversal' mode 
min_reversal_improvement = 2  # Only used in 'reversal' mode
# --- General filters ---
current_rank_bracket_start = 1 
current_rank_bracket_end = 1000

In [18]:
rank_candidates = find_rank_dip_or_reversal_opportunities(
    selected_df,
    lookback_days=lookback_days,
    recent_days=recent_days,
    min_lookback_rank_improvement=min_lookback_rank_improvement,
    # === Dip Mode Parameter ===
    min_recent_rank_drop=min_recent_rank_drop,  # <-- This activates 'Dip' mode
    # === Reversal Mode Parameters ===
    min_dip_magnitude=min_dip_magnitude,       # <-- This...
    min_reversal_improvement=min_reversal_improvement, # <-- ...and this activate 'Reversal' mode
    # === Current Rank Bracket-Filters ===    
    current_rank_bracket_start=current_rank_bracket_start,
    current_rank_bracket_end=current_rank_bracket_end,
)


Running in 'Reversal' mode...
Analyzing 985 tickers from 2025-06-09 to 2025-07-11, currently ranked between 1 and 1000...


In [31]:
# --- Display the Results Sort by the 'dist_from_best_rank' column ---
if rank_candidates:
    df_reversals = pd.DataFrame(rank_candidates)
    print(f"Found {len(df_reversals)} potential reversal opportunities:")
    # Sort by the 'dist_from_best_rank' column in ascending order (default)
    # df_sorted_asc = df_reversals.sort_values(by='dist_from_best_rank')
    sorted_df = df_reversals.sort_values(
        by=['dist_from_best_rank', 'lookback_slope'],
        ascending=[True, True]  # True for the first column, False for the second
    )    
    print(f'sorted_df.head(50):\n{sorted_df.head(50)}')


Found 133 potential reversal opportunities:
sorted_df.head(50):
    ticker  current_rank  best_rank  dist_from_best_rank  dip_rank  dip_magnitude  reversal_improvement  lookback_slope
3     CRDO           757        757                    0       781              0                    24           -8.12
4     SOFI           594        594                    0       621              0                    27           -7.00
7     WYNN           975        975                    0       986              4                    11           -5.72
8     COIN           157        157                    0       175              0                    18           -5.61
9       SN           802        802                    0       832              0                    30           -5.28
13    FLEX           679        679                    0       692              0                    13           -4.17
15      WF           870        870                    0       897             11               

In [ ]:
#############################################

In [29]:
import plotly.express as px
import pandas as pd
import math

def plot_rank_with_criteria(df_rank_history, ticker_list, title_suffix="", filter_criteria=None,
                            width=1100, height=700):
    """
    Plots rank history with buttons that correctly toggle trace visibility and legend appearance.
    - 'Clear All' sets traces to 'legendonly', hiding them and graying out the legend item.
    - 'Reset View' makes all traces fully visible.

    Args:
        df_rank_history (pd.DataFrame): The full rank history DataFrame.
        ticker_list (list): A list of ticker symbols to plot.
        title_suffix (str, optional): Text to append to the main plot title.
        filter_criteria (dict, optional): A dictionary of filter parameters to display.
        width (int, optional): The width of the figure in pixels.
        height (int, optional): The height of the figure in pixels.
    """
    if not ticker_list:
        print("Ticker list is empty. Nothing to plot.")
        return

    # Prepare data for plotting
    plot_df = df_rank_history.loc[ticker_list].T
    plot_df.index = pd.to_datetime(plot_df.index)
    
    fig = px.line(
        plot_df, 
        x=plot_df.index, 
        y=plot_df.columns,
        title=f"Rank History: {title_suffix}",
        labels={'value': 'Rank', 'x': 'Date', 'variable': 'Ticker'}
    )
    
    fig.update_yaxes(autorange="reversed")

    # --- FINAL MODIFICATION: Use `visible='legendonly'` for correct legend behavior ---
    num_traces = len(fig.data)
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.01, xanchor="left",
                y=1.1, yanchor="top",
                showactive=False,
                buttons=list([
                    # "Reset" makes all traces visible again.
                    dict(label="Reset View",
                         method="restyle",
                         args=[{"visible": [True] * num_traces}]),
                    
                    # "Clear All" sets traces to 'legendonly' mode.
                    # This hides the line on the plot but keeps the legend item, grayed out.
                    dict(label="Clear All",
                         method="restyle",
                         args=[{"visible": ['legendonly'] * num_traces}]),
                ]),
            )
        ]
    )
    
    # --- The rest of the function remains the same ---
    # ... (criteria formatting, margins, etc.) ...
    criteria_text = ""
    num_rows = 0
    if filter_criteria:
        active_criteria = {k: v for k, v in filter_criteria.items() if v is not None}
        col_width = 38
        criteria_lines = []
        items = list(active_criteria.items())
        
        for i in range(0, len(items), 3):
            chunk = items[i:i+3]
            line_parts = [f"{f'  • {k}: {v}':<{col_width}}" for k, v in chunk]
            criteria_lines.append("".join(line_parts))

        num_rows = len(criteria_lines)
        criteria_text = "<b>Filter Criteria:</b><br>" + "<br>".join(criteria_lines)

    if criteria_text:
        fig.add_annotation(
            showarrow=False, text=criteria_text, xref="paper", yref="paper",
            x=0, y=-0.15, xanchor="left", yanchor="top", align="left",
            font=dict(family="Courier New, monospace", size=11)
        )
    
    bottom_margin = 80 + (num_rows * 18)
    fig.update_layout(
        width=width, height=height, margin=dict(b=bottom_margin)
    )
        
    fig.show()
    return fig

In [30]:
# 1. Call the analysis function and get both the candidates and the criteria
# Using 'dip' mode for this example
rank_candidates, filter_criteria = find_rank_dip_or_reversal_opportunities(
    selected_df,
    lookback_days=lookback_days,
    recent_days=recent_days,
    min_lookback_rank_improvement=min_lookback_rank_improvement,
    # === Dip Mode Parameter ===
    min_recent_rank_drop=min_recent_rank_drop,  # <-- This activates 'Dip' mode
    # === Reversal Mode Parameters ===
    min_dip_magnitude=min_dip_magnitude,       # <-- This...
    min_reversal_improvement=min_reversal_improvement, # <-- ...and this activate 'Reversal' mode
    # === Current Rank Bracket-Filters ===    
    current_rank_bracket_start=current_rank_bracket_start,
    current_rank_bracket_end=current_rank_bracket_end,
)

# 2. Convert the list of dicts to a DataFrame for easy sorting and slicing
df_candidates = pd.DataFrame(rank_candidates)

# 3. Get the top N tickers you want to plot
tickers_to_plot = df_candidates.head(10)['ticker'].tolist()

# 4. Call the new plotting function
if tickers_to_plot:
    plot_rank_with_criteria(
        df_rank_history=selected_df,
        ticker_list=tickers_to_plot,
        title_suffix="'Dip' Candidates",
        filter_criteria=filter_criteria # Pass the dictionary here
    )
else:
    print("No candidates found to plot.")

Running in 'Reversal' mode...
Analyzing 985 tickers from 2025-06-09 to 2025-07-11, currently ranked between 1 and 1000...
